problem: https://adventofcode.com/2024/day/12

data: https://adventofcode.com/2024/day/12

In [1]:
import numpy as np

DIRECTIONS = (
    (1, 0),
    (0, 1),
    (-1, 0),
    (0, -1)
)

In [2]:
rows = []
with open('problem12_data.txt', 'r') as file:
  for row in file:
    rows.append(list(row.strip()))
data = np.array(rows)

In [3]:
data

array([['T', 'T', 'T', ..., 'H', 'H', 'H'],
       ['T', 'T', 'T', ..., 'H', 'H', 'H'],
       ['T', 'T', 'T', ..., 'H', 'H', 'H'],
       ...,
       ['T', 'T', 'T', ..., 'Q', 'Q', 'Q'],
       ['T', 'T', 'T', ..., 'Q', 'Q', 'Q'],
       ['T', 'T', 'T', ..., 'Q', 'Q', 'Q']], dtype='<U1')

In [4]:
df = data[10:20, 10:20]
print(df)

[['B' 'D' 'X' 'D' 'X' 'X' 'X' 'X' 'X' 'X']
 ['B' 'D' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'G']
 ['B' 'B' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['B' 'B' 'B' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['B' 'B' 'C' 'X' 'X' 'X' 'X' 'O' 'O' 'O']
 ['B' 'X' 'X' 'X' 'X' 'X' 'X' 'O' 'O' 'O']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'O' 'O' 'O']
 ['F' 'X' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O']
 ['X' 'X' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O']
 ['A' 'A' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O']]


In [5]:
def add_tuples(x, y) -> tuple:
    return (x[0] + y[0], x[1] + y[1])

def tuple_is_valid(matrix, x):
    return x[0] >= 0 and x[1] >= 0 and x[0] < matrix.shape[0] and x[1] < matrix.shape[1]

In [6]:
def spot_region(garden, start: tuple[int]) -> list[tuple[int]]:
    plant_type = garden[start]
    result = []
    visited = []
    to_be_checked = [start]
    while len(to_be_checked) > 0:
        plot = to_be_checked.pop(0)
        visited.append(plot)
        if garden[plot] == plant_type:
            result.append(plot)
            for direction in DIRECTIONS:
                new_plot = add_tuples(plot, direction)
                if tuple_is_valid(garden, new_plot) and new_plot not in visited and new_plot not in to_be_checked:
                    to_be_checked.append(new_plot)
    return result

In [7]:
def region_perimeter(garden, region) -> int:
    result = 0
    plant_type = garden[region[0]]
    for plot in region:
        for direction in DIRECTIONS:
            adiacent_plot = add_tuples(plot, direction)
            if not tuple_is_valid(garden, adiacent_plot):
                result += 1
            elif garden[adiacent_plot] != plant_type:
                result += 1
            else:
                continue
    return result

In [8]:
regions = []
perimeters = []
visited = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if (i, j) not in visited:
            region = spot_region(data, (i, j))
            regions.append(region)
            visited.extend(region)
            perimeters.append(region_perimeter(data, region))

solution = 0
for r, p in zip(regions, perimeters):
    solution = solution + len(r) * p
print(f'{solution = }')

solution = 1518548


**Part 2**

In [21]:
def region_perimeter(garden, region) -> list[tuple[tuple]]:
    result = []
    plant_type = garden[region[0]]
    for plot in region:
        for direction in DIRECTIONS:
            adiacent_plot = add_tuples(plot, direction)
            if not tuple_is_valid(garden, adiacent_plot):
                result.append((plot, adiacent_plot))
            elif garden[adiacent_plot] != plant_type:
                result.append((plot, adiacent_plot))
            else:
                continue
    return result

def rotate_direction(d) -> tuple:
    return (d[1], -d[0])

def contiguous_sides(a, b, directions=DIRECTIONS) -> bool:
    a1, a2 = a
    b1, b2 = b
    side_direction_1 = rotate_direction((a1[0] - a2[0], a1[1] - a2[1]))
    side_direction_2 = (-side_direction_1[0], -side_direction_1[1])
    for d in (side_direction_1, side_direction_2):
        if b1 == add_tuples(a1, d) and b2 == add_tuples(a2, d):
            return True
    return False

def stick_sides(sides):
    if len(sides) < 2:
        return sides
    result = sides.copy()
    go = True
    while go:
        go = False
        for i in range(len(result)-1):
            x = result[i]
            for j in range(i+1, len(result)):
                y = result[j]
                if contiguous_sides(x[-1], y[0]):
                    x.extend(y)
                    go = True
                    result.remove(y)
                    break
                elif contiguous_sides(y[-1], x[0]):
                    y.extend(x)
                    go = True
                    result.remove(x)
                    break
    return result

def perimeter_sides(perimeter) -> int:
    sides = []
    for p in perimeter:
        if len(sides) == 0:
            sides.append([p])
        else:
            inserted = False
            for s in sides:
                if contiguous_sides(s[0], p):
                    s.insert(0, p)
                    inserted = True
                    break
                elif contiguous_sides(s[-1], p):
                    s.append(p)
                    inserted = True
                    break
            if not inserted:
                sides.append([p])
    return len(stick_sides(sides))

In [24]:
regions = []
perimeters = []
visited = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if (i, j) not in visited:
            region = spot_region(data, (i, j))
            regions.append(region)
            visited.extend(region)
            perimeters.append(region_perimeter(data, region))

solution = 0
for r, p in zip(regions, perimeters):
    solution = solution + len(r) * perimeter_sides(p)
print(f'{solution = }')

solution = 909564
